This is an attempt at re-implementing BERTForTokenClassification to figure out why loss is stagnant in the real model.

Here, the task is simplified to tagging words as nouns or non-nouns.

In [9]:
import torch
import transformers
from transformers import BertTokenizerFast
from torch.nn.utils.rnn import pad_sequence

In [5]:
training_sentences = [
    "the dog jumped over the cat .",
    "cats are cool .",
    "the ocean contains much water .",
    "the sky is blue ."
]
training_labels = [
    [0, 1, 0, 0, 0, 1, 0],
    [1, 0, 0, 0],
    [0, 1, 0, 0, 1, 0],
    [0, 1, 0, 0, 0]
]

In [ ]:
test_sentences = []

In [7]:
device = torch.device('cuda')

In [8]:
class TokenModel(torch.nn.Module):
    def __init__(self):
        self.bert = transformers.BertModel.from_pretrained("bert-base-uncased")
        self.linear = torch.nn.Linear(768, 1).to(device)
        
    def forward(self, ids, mask, token_type_ids):
        padded_ids = pad_sequence(ids)
        padded_ids = padded_ids.permute(1, 0, 2).to(self.device)
        padded_masks = pad_sequence(mask)
        padded_masks = padded_masks.permute(1, 0, 2).to(self.device)
        padded_token_type_ids = pad_sequence(token_type_ids)
        padded_token_type_ids = padded_token_type_ids.permute(1, 0, 2).to(self.device)
        
        results = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)
        last_hidden_state = results["last_hidden_state"]
        

In [31]:
loss_func = torch.nn.BCEWithLogitsLoss()

In [20]:
targets

tensor([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [39]:
outputs = torch.randn((18))
targets = torch.zeros(18)
targets[3] = 1
targets[8] = 1


In [40]:
loss_func(outputs, targets)

tensor(0.7861)

In [18]:
outputs

tensor([[ 0.7014],
        [ 0.6054],
        [-0.1036],
        [-0.4904],
        [ 1.6244],
        [-1.4295],
        [ 1.4354],
        [ 0.3189],
        [-1.2940],
        [ 0.0971],
        [-1.0361],
        [-0.6049],
        [ 0.0518],
        [-1.3608],
        [ 1.3989],
        [-0.8583],
        [-1.2326],
        [ 0.8343]])